In [1]:
import numpy as np, os, random, json, re
from PIL import Image
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
# sort files as 1,2,3,4,... and not as 1,10,100,11,...
def sort_nicely(l):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    l.sort( key=alphanum_key )

In [3]:
dir_1 = # where the openpose parameters of kostas' images are saved
dir_2 = # where the openpose parameters of olga's  images are saved

classes = np.load('classes.npy')

In [4]:
x, y_n = [], []
for c in classes:
    for dir in os.listdir(dir_1):
        if c==dir:
            x.extend([dir_1+dir+'/'+f for f in os.listdir(dir_1+dir) if 'npz' not in f])
            length = len([f for f in os.listdir(dir_1+dir) if 'npz' not in f])
            y_n.extend([dir]*length)
            
for c in classes:
    for dir in os.listdir(dir_2):
        if c==dir:
            x.extend([dir_2+dir+'/'+f for f in os.listdir(dir_2+dir) if 'npz' not in f])
            length = len([f for f in os.listdir(dir_2+dir) if 'npz' not in f])
            y_n.extend([dir]*length)

In [5]:
le = LabelEncoder()
y = le.fit_transform(y_n)
x = np.array(x)

In [6]:
X_train, X_dev, X_test = [], [], []
Y_train, Y_dev, Y_test = [], [], []
for i in range(0,max(y)+1):
    indices = [j for j in range(len(y)) if y[j]==i]; random.shuffle(indices)
    train = indices[0:int(len(indices)*0.6)]
    dev   = indices[int(0.6*len(indices)):int(0.8*len(indices))]
    test  = indices[int(0.8*len(indices)):len(indices)]
    X_train.extend(x[train]); Y_train.extend([i]*len(train))
    X_dev.extend(x[dev]); Y_dev.extend([i]*len(dev))
    X_test.extend(x[test]); Y_test.extend([i]*len(test))

In [7]:
train_feat, i = [], 0
for x in X_train:
    features = []
    jsons = [x+'/'+f for f in os.listdir(x)]; sort_nicely(jsons)
    for j in jsons:
        with open(j) as u:
            data = json.load(u)['people']
            if len(data)==0: continue
            x1 = data[0]['pose_keypoints_2d']
            x2 = data[0]['face_keypoints_2d']
            x3 = data[0]['hand_left_keypoints_2d']
            x4 = data[0]['hand_right_keypoints_2d']
            features.append(x1+x3+x4)
            #features.append(x1+x3+x4)
    if len(features)!=len(jsons): print(len(features),len(jsons))
    if len(features)<5: del Y_train[i]; print(x)
    else: i+=1; train_feat.append(features)

dev_feat, i = [], 0
for x in X_dev:
    features = []
    jsons = [x+'/'+f for f in os.listdir(x)]; sort_nicely(jsons)
    for j in jsons:
        with open(j) as u:
            data = json.load(u)['people']
            if len(data)==0: continue
            x1 = data[0]['pose_keypoints_2d']
            x2 = data[0]['face_keypoints_2d']
            x3 = data[0]['hand_left_keypoints_2d']
            x4 = data[0]['hand_right_keypoints_2d']
            features.append(x1+x3+x4)
    if len(features)!=len(jsons): print(len(features),len(jsons))
    if len(features)<5: del Y_dev[i]; print(x)
    else: i+=1; dev_feat.append(features)

test_feat, i = [], 0
for x in X_test:
    features = []
    jsons = [x+'/'+f for f in os.listdir(x)]; sort_nicely(jsons)
    for j in jsons:
        with open(j) as u:
            data = json.load(u)['people']
            if len(data)==0: continue
            x1 = data[0]['pose_keypoints_2d']
            x2 = data[0]['face_keypoints_2d']
            x3 = data[0]['hand_left_keypoints_2d']
            x4 = data[0]['hand_right_keypoints_2d']
            features.append(x1+x3+x4)
    if len(features)!=len(jsons): print(len(features),len(jsons))
    if len(features)<5: del Y_test[i]; print(x)
    else: i+=1; test_feat.append(features)

In [8]:
scaler = StandardScaler()

for i in range(len(train_feat)):
    train_feat[i] = scaler.fit_transform(train_feat[i])
    
for i in range(len(dev_feat)):
    dev_feat[i] = scaler.fit_transform(dev_feat[i])
    
for i in range(len(test_feat)):
    test_feat[i] = scaler.fit_transform(test_feat[i])

In [9]:
np.save('data/x_dev_op.npy',dev_feat); np.save('data/y_dev_op.npy',Y_dev)
np.save('data/x_test_op.npy',test_feat); np.save('data/y_test_op.npy',Y_test)
np.save('data/x_train_op.npy',train_feat); np.save('data/y_train_op.npy',Y_train)